<a href="https://colab.research.google.com/github/GuillermoSego/TSF/blob/main/OpenA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cómo leer un archivo csv en C

**FILE** es la estructura que utiliza en C para la administración de los archivos. Se utiliza un apuntador hacia el archivo. Cada archivo que se abre necesita un apuntador de tipo *FILE*. Posteriormente el archivo se abre utilizando la instrucción **fopen**. 

Otro concepto que es necesario para abrir un archivo es el llamado **bufer**. Un buffer es un espacio de almacenamiento temporal. Todos los dispositivos de entrada poseen un buffer. Cuando nosotros queremos pasar mas de un valor de entrada de los requeridos, los valores restantes se almacenan en el *buffer*. Estos datos van automáticamente al siguiente input si este existe. 

Algunas otras funciones de C que es necesario recordar:
* **fgets**: Recibe como argumento el apuntador tipo FILE del archivo cuyos datos se quieran leer. 

```
fgets(char *str, int n, FILE *stream)
```

**str**: Este es el apuntador del vector de caracteres donde se almacena la lectura de la cadena, en este caso el buffer.

**n**: Máximo número de caracteres por leer. Tamaño del buffer.

**stream**: Apuntador al objeto tipo FILE.

* **sscanf**: Lee la entrada formateada de una cadena de caracteres.


```
sscanf(const char *str, const char *format, ...)
```
**str**: Es la cadena de caracteres que se procesa como entrada para recuperar los datos. 

**format**: Esta es la cadena contiene uno o más de los siguientes elementos: espacios en blanco o saltos de línea y especificadores de formato.

**Otros argumentos**: Esta función espera una secuencia de apuntadores como argumentos adicionales, cada uno apuntando a un objeto del tipo especificado por su etiqueta % correspondiente dentro de la cadena de formato, en el mismo orden. 

Para cada especificador de formato en la cadena de formato que recupera datos, se debe especificar un argumento adicional. Si desea almacenar el resultado de una operación sscanf en una variable, debe preceder su identificador con el operador de referencia, es decir, (&), como: 

```
int n; 
sscanf(str,"%d",&n);
```


In [63]:
%%writefile n.c

#include <stdio.h>

#define BF 100

// Manejador de errores

void ErrorManagement(int e)
{
    if (e == 0) {
        
        printf("Error al abrir el archivo\n");
    }

}

// Imprimir una matriz

void MatrixShow(int m, int n, double v[])
{
    
     
    int i, j, k=0;
    
    for(i=0 ; i<m ; i++){
        
        for (j=0 ; j<n ; j ++){
            
            printf("%lf \t", v[k]);
            k ++;
        }
        
        printf("\n");
    }
       
}

// Esta rutina calcula la matriz transpuesta de un vector y la guarda en otro

void MatrixT(int n, int m, double v[], double* VecPtr )
{

  double* vPtr = v;

  int i = 0, j = 0;
 
  do {

    do {
        
       *VecPtr = *vPtr;

        VecPtr ++; 

        vPtr += m;

      j ++;
    } while (j<n);   

    vPtr = v + (i+1);

    j = 0;
    i ++;
  } while (i<m);


}

int main(int nArg, char *arg[])
{

  // Declaramos una variable tipo FILE
  FILE* Doc;

  // Declaramos un vector en el que vamos a guardar los datos
  double DatV[BF];  
 
  // Abrimos el archivo. "r": Abrir un archivo existente para leer. 
  // Hacemos que el apuntador Doc apunte en la direccion de los datos
  Doc = fopen ( arg[1], "r" );
 
  // Manejamos el error: no se pueda abrir el archivo
  if (Doc == NULL)
  {
      ErrorManagement( 0 );
      return 0;
  } 
 
  // Continuamos con la estructura para leer los datos
 
  // Primero creamos un buffer para guardar los datos
	char buffer[BF];

  // Se lee la primera linea y se almacena en el buffer como variable char
	fgets(buffer, BF, Doc);
  
  /* 
 
    En este punto todo esta almacenado en el buffer como una sola cadena de 
    caracteres. Por lo que para poder acceso a los datos se tiene que convertir
    esta cadena a binario, separando por comas "," ya que un archivo tipo csv
    separa los datos por comas.
 
  */
  
  // Creamos un apuntador que apunte a la cadena de caracteres
  char* BfPtr = buffer;
 
  // Calculamos el número de columnas contando las comas de la primera linea
  int column = 0;
    
  do{
      column += ( *BfPtr == ',' );

  } while( *BfPtr ++);
 
  // Convertimos a binario
 
  // Apuntamos en la direccion del buffer
  BfPtr = buffer;
 
  // Apuntador al vector de datos
  double* DatPtr = DatV;
  
  // Creamos un contador igual al número de columnas
  // Declaramos el parámetro adicional para la función sscanf
  int i = column, length;
 
  // Leemos el primer renglon 

  do{
        
      BfPtr += length;
      sscanf( BfPtr, "%lf,%n", DatPtr ++, &length );
        
    } while(i--);

  /*
  
    En este punto en el vector DatV tenemos guardado la primer fila de los datos.
    Ahora tenemos que guardar las siguientes filas.
 
    El apuntador del buffer BfPtr se encuentra apuntando hacia el primer dato 
    de la última columna, para asegurarnos de que apuntamos exactamente al ultimo
    elemento de la primera fila, le sumamos la longitud length

  */
  BfPtr += length;

  /*
 
  Leemos las siguientes lineas del buffer y las ponemos en el apuntador 
  de los datos

  */
  sscanf( BfPtr, "%lf", DatPtr);  

  // Recorremos el apuntador de los datos
  i = column;
  DatPtr = DatV;

	do {
		*DatPtr ++;
	} while( i-- );
		
	// Se termina de leer el archivo
	
  /*
 
  En este momento leemos los demas datos del apuntador Doc, y los guardamos en
  el apuntador de los datos DatPtr, sin embargo, tenemos que llegar al final de
  los datos. Esto lo hacemos creando una variable llamada dummy en la que guar-
  damos el momento en el que se llega al final, es decir cuando tenemos que los 
  datos tienen la forma "%lf%c"

 */
  
  int Error = 0, col = column+1; 
  char dummy;

	do{

		Error = fscanf(Doc, "%lf%c", DatPtr++, &dummy);
         
	} while(Error > 0);


  // Cerramos el archivo, es muy importante cerrar el archivo
  fclose ( Doc );

  int SA, ST;
  
  SA = (DatPtr - DatV)/ (col);
  ST = col;

  printf("El numero de columnas: %d\n", ST);
  printf("El numero de filas: %d\n", SA);
 
  // Imprimimos la matriz de los datos
  MatrixShow( SA, ST, DatV );
  
  // Declaramos un nuevo vector en el que guardaremos la transpuesta 
  double Vec[ST*SA];
  double* VecPtr = Vec;

  MatrixT(SA, ST, DatV, VecPtr);

  return 0;


}

Overwriting n.c


In [68]:
%%shell

gcc n.c -o output
./output Datos.csv

El numero de columnas: 5
El numero de filas: 15
10.000000 	10.700000 	10.700000 	10.000000 	10.700000 	
10.400000 	9.800000 	9.800000 	10.400000 	9.800000 	
9.700000 	10.000000 	10.000000 	9.700000 	10.000000 	
9.700000 	10.100000 	10.100000 	9.700000 	10.100000 	
11.700000 	11.500000 	11.500000 	11.700000 	11.500000 	
11.000000 	10.800000 	10.800000 	11.000000 	10.800000 	
8.700000 	8.800000 	8.800000 	8.700000 	8.800000 	
9.500000 	9.300000 	9.300000 	9.500000 	9.300000 	
10.100000 	9.400000 	9.400000 	10.100000 	9.400000 	
9.600000 	9.600000 	9.600000 	9.600000 	9.600000 	
10.500000 	10.400000 	10.400000 	10.500000 	10.400000 	
9.200000 	9.000000 	9.000000 	9.200000 	9.000000 	
11.300000 	11.600000 	11.600000 	11.300000 	11.600000 	
10.100000 	9.800000 	9.800000 	10.100000 	9.800000 	
8.500000 	9.200000 	9.200000 	8.500000 	9.200000 	
